## Download data

Used pandas dataframe to contain all data collected in "All_publications_fin_

# Data processing

Use pandas dataframe to contain all data collected from data collection into the file "All_data_fin_Scopus+Scimago.xlsx"

### Packages, classes and functions used:
**pandas package**
- .DataFrame()
- .read_excel()
- .value_counts()
- .sort_values()
- .reset_index()
- .rename()
- .to_csv()
- .groupby()
- .copy()
- .pivot_table()
- .melt()
- .iterrows()

**requests**
- .get()

**inflect**
- .engine()
- .singular_noun()

In [1]:
#!pip install inflect
#!pip install requests

Download all data from the file "All_data_fin_Scopus+Scimago.xlsx" created during data collection.

In [2]:
import pandas as pd

file_path = "All_data_fin_Scopus+Scimago.xlsx"
df = pd.read_excel(file_path, index_col = 0)
df

,eid,doi,subtypeDescription,coverDate,publicationName,aggregationType,volume,issueIdentifier,pageRange,citedby_count,...,Total Docs. (2014),Total Docs. (2015),Total Docs. (2016),Total Docs. (2017),Total Docs. (2018),Total Docs. (2019),Total Docs. (2020),Total Docs. (2021),Total Docs. (2022),Title
1,2-s2.0-84891116423,NaN,Article,2013-12-31,International Journal of Intangible Heritage,Journal,8.0,NaN,19-36,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2-s2.0-84890442458,10.1007/s10745-013-9614-8,Article,2013-12-01,Human Ecology,Journal,41.0,6,905-914,128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2-s2.0-84887148964,10.1007/s10668-013-9446-0,Article,2013-12-01,"Environment, Development and Sustainability",Journal,15.0,6,1477-1494,84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2-s2.0-84897839499,10.1080/1533015X.2013.876250,Article,2013-10-01,Applied Environmental Education and Communication,Journal,12.0,4,224-234,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2-s2.0-84884132626,10.1177/1075547012470821,Article,2013-10-01,Science Communication,Journal,35.0,5,572-602,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,2-s2.0-85118639900,10.1080/10509208.2021.1996310,Article,2023-01-01,Quarterly Review of Film and Video,Journal,40.0,2,187-214,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Role of Forest and Environmental Conservat...
675,2-s2.0-85114873429,10.1080/02635143.2021.1978421,Article,2023-01-01,Research in Science and Technological Education,Journal,41.0,3,961-982,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teaching and environmentalism: a deduction fro...
676,2-s2.0-85108819834,10.1080/14729679.2021.1935284,Article,2023-01-01,Journal of Adventure Education and Outdoor Lea...,Journal,23.0,1,25-37,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Autoethnographic stories for self and environm...
677,2-s2.0-85106234545,10.1080/02508281.2021.1920755,Article,2023-01-01,Tourism Recreation Research,Journal,48.0,3,399-418,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Environmental attitudes and behaviour of birdw...


### Publication count for every year
Find the number of publications by year, by utilizing value_count() and counting occurences in the column "year". This data is then stored in the "Data_for_visualization" folder in a file called *"Count_yearly_publications.csv"*.

In [9]:
year_counts = df['year'].value_counts()
year_counts.index.name = 'count'
# Reset the index to turn 'year' into a regular column
year_counts = year_counts.reset_index()

# Rename the 'year' column to 'count'
year_counts = year_counts.rename(columns={'year': 'count', "count":"year"})
year_counts = year_counts.sort_values("year").reset_index()

year_counts.to_csv('./Data_for_visualization/Count_yearly_publications.csv', header=True, index_label='index')

### Quartile count for every year

Find the number of the four different quartiles for every year. As not all years have an entry for every quartile a pivot table is used to fill in any missing quartile values with 0. This is done to easily be able to visualize the data later. 
Finally the data is sorted by year and quartile, before being saved to the file called *"Count_quartile_year"*.

In [4]:
quartile_count_df = df.groupby(['year', 'SJR Best Quartile']).size().reset_index(name='Count')
quartile_count_df_filtered = quartile_count_df[quartile_count_df['SJR Best Quartile'] != '-']

quartile_count_df_filtered = quartile_count_df_filtered.copy()
quartile_count_df_filtered.rename(columns={"SJR Best Quartile": "quartile"}, inplace=True)

# Create a pivot table to fill in missing quartile with zeros
quartile_count_df_pivot = quartile_count_df_filtered.pivot_table(index='year', columns='quartile', values='Count', fill_value=0).reset_index()

# Reshape the DataFrame for consistency
quartile_count_df_pivot = quartile_count_df_pivot.melt(id_vars='year', value_name='Count')

quartile_count_final = quartile_count_df_pivot.sort_values(["year", "quartile"]).reset_index(drop=True)

quartile_count_final.to_csv('./Data_for_visualization/Count_quartile_year', header=True, index_label='index')

### Keyword counts

Finding keyword count by itterating over the all keywords and recording the count of them in a dictionary. Firstly the keyword string is made into a list of strings by splitting on ";".

As some of the keywords are written in plural form and some in singular form, the python package *inflect* is utilized to make all words into singular form.

Furthermore some of the spellings of the keywords are American, while otheres are British. To still count these words as the same keyword, a dictionary containg the american and the british spellin is utilized. In this way all keywords are written in American english, making so that there are not duplicates.

A dictonary of keyword_count is updated with the count of all keywords. This is saved to a dataframe, and sorted by the count. As the number of keywords is very large, only the keywords with a count larger than 6 is kept for use in visualization. 

The keyword and counts are saved to the file *"keyword_count.csv"*.

In [5]:
# Split the 'Author Keywords' column by ';'
df['Author Keywords'] = df['Author Keywords'].str.split(';')

In [6]:
import requests
import inflect
p = inflect.engine()
url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
british_to_american_dict = requests.get(url).json()   

keyword_counts = {}
for index, row in df.iterrows():
    keywords = row['Author Keywords']
    if isinstance(keywords, list):
        for keyword in keywords:
            keyword = keyword.strip().lower()
            keyword_singular = p.singular_noun(keyword)
            if keyword_singular != False:
                keyword=keyword_singular
            for british_spelling, american_spelling in british_to_american_dict.items():
                keyword= keyword.replace(british_spelling, american_spelling)
            keyword_counts[keyword] = keyword_counts.get(keyword, 0) + 1

# Create a DataFrame from the dictionary
df_keyword_counts = pd.DataFrame(list(keyword_counts.items()), columns=['keyword', 'count'])
df_keyword_counts_sorted = df_keyword_counts.sort_values(by='count', ascending=False)
df_keyword_counts_sorted = df_keyword_counts_sorted.reset_index(drop=True)
df_keyword_counts_filterd = df_keyword_counts_sorted[df_keyword_counts_sorted['count'] > 6]
df_keyword_counts_filterd.to_csv('./Data_for_visualization/keyword_count.csv', header=True)